# Empirical Project 2 - Working in Python

## Getting started in Python

TODO

## Part 2.1 Collecting data by playing a public goods game

### Python Walkthrough 2.1

**Plotting a line chart with multiple variables**

Use the data from your own experiment to answer Question 1. As an example, we will use the data for the first three cities of the dataset that will be introduced in Part 2.2.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Set the plot style for prettier charts:
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams["figure.dpi"] = 150

# Create a dictionary with the data in
data = {"Copenhagen": [14.1, 14.1, 13.7, 12.9, 12.3, 11.7, 10.8, 10.6, 9.8, 5.3],
        "Dniprop": [11.0, 12.6, 12.1, 11.2, 11.3, 10.5, 9.5, 10.3, 9.0, 8.7],
        "Minsk": [12.8, 12.3, 12.6, 12.3, 11.8, 9.9, 9.9, 8.4, 8.3, 6.9]}

df = pd.DataFrame.from_dict(data)
df.head()

In [ ]:
# Plot the data
fig, ax = plt.subplots()
df.plot(ax=ax)
ax.set_title("Average contribution to public goods game: without punishment")
ax.set_ylabel("Average contribution")
ax.set_xlabel("Round");


## Part 2.2 Describing the data

### Python Walkthrough 2.2

Both the tables you need are in a single Excel worksheet. Note down the cell ranges of each table, in this case A2:Q12 for the without punishment data and A16:Q26 for the punishment data. We will now use this range information to import the data into two dataframes (data_n and data_p respectively).

In [ ]:
data_np = pd.read_excel("data/Public-goods-experimental-data.xlsx", usecols="A:Q", header=1, index_col="Period")
data_n = data_np.iloc[:10, :].copy()
data_p = data_np.iloc[14:24, :].copy()

Look at the data either by opening the dataframes from the Environment window or by typing data_n or data_p into the interactive Python window.

You can see that in each row, the average contribution varies across countries; in other words, there is a distribution of average contributions in each period.

### Python Walkthrough 2.3

**Calculating the mean using different methods**

We calculate the mean using two different methods, to illustrate that there are usually many ways of achieving the same thing. We apply the first method on `data_n`, which uses the built-in `.mean()` function to calculate the average separately over each column except the first. We use the second method (the agg function) on `data_p`.

In [ ]:
data_n["mean_c"] = data_n.mean(axis=1)
data_p["mean_c"] = data_p.agg(np.mean, axis=1)

As the name suggests, the `agg` function applies an aggregation function (the mean function in this case) to all rows or columns in a dataframe. The second input, `axis=1`, applies the specified function to all rows in data_p. Typing 0 would have calculated column means instead (check and see for yourself). Type `help(pd.DataFrame.agg)` in your interactive Python window for more details, or see Python Walkthrough 2.5 for further practice.

**Plot the mean contribution**

Now we will produce a line chart showing the mean contributions.

In [ ]:
fig, ax = plt.subplots()
data_n["mean_c"].plot(ax=ax, label="Without punishment")
data_p["mean_c"].plot(ax=ax, label="With punishment")
ax.set_title("Average contribution to public goods game")
ax.set_ylabel("Average contribution")
ax.legend();

In [ ]:
# code

The difference between experiments is stark, as the contributions increase and then stabilize at around $13 when there is punishment, but decrease consistently from around $11 to $4 across the rounds when there is no punishment.

### Python Walkthrough 2.4

**Drawing a column chart to compare two groups**

To make a column chart, we will use the `.plot.bar()` function. We first extract the four data points we need (Periods 1 and 10, with and without punishment) and place them into another dataframe.

In [ ]:

# code
df.plot.bar()

In [ ]:
# code

*Tip*: Experimenting with these charts will help you to learn how to use Python and its packages. The details of how to specify the column chart may look complicated, but you can see from Figure 2.3 what the options [TO DO] do.

### Python Walkthrough 2.5

**Calculating and understanding the standard deviation**

In order to calculate these standard deviations and variances, we will use the agg function, which we introduced in Python Walkthrough 2.3. As we saw, `agg` is a command asking **pandas** to aggregate a set of rows or columns of the dataframe using a particular aggregation function. The basic structure is as follows: `df.loc[conditions or rows, columns].agg([function1, function2, ...])`. So to calculate the variances, we use the following command:

In [ ]:
# code

Here we take `data_n.iloc[:, 2:17]` and apply the var function to each row (recall that the second input 0 does this; 1 would indicate columns). Note that as in Python Walkthrough 2.3, we exclude the first column from the calculation, as that contains the period numbers. The result is saved as a new variable called `var_c`.

We then apply the same principle to the standard deviation calculation and the `data_p` dataframe.

In [ ]:
# code

To determine whether 95% of the observations fall within two standard deviations of the mean, we can use a line chart. As we have 16 countries in every period, we would expect about one observation (0.05 × 16 = 0.8) to fall outside this interval.

In [ ]:
# code

None of the observations fall outside the mean ± two standard deviations interval for the public goods game without punishment. Let’s see the equivalent chart for the version with punishment.